# Grasp annotation example

In [ ]:
from pathlib import Path
import numpy as np
import open3d as o3d
import rerun as rr
from airo_camera_toolkit.point_clouds.conversions import open3d_to_point_cloud, point_cloud_to_open3d
from airo_camera_toolkit.point_clouds.operations import filter_point_cloud
from cloth_tools.dataset.format import load_competition_input_sample
from pydrake.math import RigidTransform, RollPitchYaw
import matplotlib.pyplot as plt
from cloth_tools.dataset.format import load_competition_input_sample
from airo_camera_toolkit.utils.image_converter import ImageConverter
from airo_typing import OpenCVIntImageType
import cv2

data_dir = Path("../data")
dataset_dir = data_dir / "competition_dev"

In [ ]:
sample = load_competition_input_sample(dataset_dir, sample_index=0)

In [ ]:
plt.figure(figsize=(20, 10))
plt.subplot(1, 2, 1)
plt.imshow(sample.image_left);

In [ ]:
confidence_map = sample.confidence_map
point_cloud_in_camera = sample.point_cloud

pcd_in_camera = point_cloud_to_open3d(point_cloud_in_camera) # X_C_PC, need X_W_C

# TODO change how camera_pose is saved
# TODO clarify somewhere what the world frame is to simplify all these transformations
camera_pose = sample.camera_pose # X_LCB_C (camera pose in the left-arm base frame)
X_LCB_C = camera_pose

y_distance = 0.45
X_W_L = RigidTransform(rpy=RollPitchYaw([0, 0, np.pi / 2]), p=[0, y_distance, 0]).GetAsMatrix4()
X_W_R = RigidTransform(rpy=RollPitchYaw([0, 0, np.pi / 2]), p=[0, -y_distance, 0]).GetAsMatrix4()
X_CB_B = RigidTransform(rpy=RollPitchYaw([0, 0, np.pi]), p=[0, 0, 0]).GetAsMatrix4()
X_LCB_W = X_CB_B @ np.linalg.inv(X_W_L)
X_RCB_W = X_CB_B @ np.linalg.inv(X_W_R)
X_W_C = np.linalg.inv(X_LCB_W) @ X_LCB_C

pcd = pcd_in_camera.transform(X_W_C) # transform to world frame
point_cloud = open3d_to_point_cloud(pcd)

confidence_threshold = 1.0
confidence_mask = (confidence_map <= confidence_threshold).reshape(-1)  # Threshold and flatten
point_cloud_filtered = filter_point_cloud(point_cloud, confidence_mask)

In [ ]:
pcd_filtered = point_cloud_to_open3d(point_cloud_filtered)

resolution = sample.camera_resolution
intrinsics = sample.camera_intrinsics

X = np.array([0, -1, 0])
Z = np.array([0, 0, -1])
Y = np.cross(Z, X)
orientation = np.column_stack([X, Y, Z])

position = np.array([0, 0, 1.5])

camera_pose_top_down = np.identity(4)
camera_pose_top_down[:3, :3] = orientation
camera_pose_top_down[:3, 3] = position

# Can be removed once the saved point clouds are float32
pcd_filtered.point.positions = o3d.core.Tensor(pcd_filtered.point.positions.numpy().astype(np.float32))
pcd_filtered.point.colors = o3d.core.Tensor(pcd_filtered.point.colors.numpy().astype(np.float32)) / 255.0

camera_pose_top_down = camera_pose_top_down
pcd_filtered.point.positions.dtype

In [ ]:
rgbd_image = pcd_filtered.project_to_rgbd_image(
    *resolution, sample.camera_intrinsics, extrinsics=np.linalg.inv(camera_pose_top_down)
)

image_rgb_float = np.asarray(rgbd_image.color)
depth_map_float = np.asarray(rgbd_image.depth).squeeze()

image_rgb_float[depth_map_float == 0.0] = 0.4

plt.figure(figsize=(20, 10))
plt.imshow(image_rgb_float)
plt.show()

In [ ]:
rr.init("Grasp Annotation - Point cloud", spawn=True)

rr_point_cloud = rr.Points3D(positions=point_cloud_filtered.points, colors=point_cloud_filtered.colors)
rr.log("world/point_cloud", rr_point_cloud)

In [ ]:
from airo_camera_toolkit.pinhole_operations.unprojection import unproject_onto_depth_values
from airo_camera_toolkit.pinhole_operations.projection import project_points_to_image_plane
from airo_spatial_algebra import transform_points

from cloth_tools.visualization.opencv import draw_pose


line_type = cv2.LINE_AA
point_size = 5
point_thickness = 3

window_frontal = "Grasp Annotation - Frontal view"
window_topdown = "Grasp Annotation - Topdown view"
windows = [window_frontal, window_topdown]

h = 600
w = int(1.5 * h)
cv2.namedWindow(window_frontal, cv2.WINDOW_NORMAL)
cv2.namedWindow(window_topdown, cv2.WINDOW_NORMAL)
cv2.resizeWindow(window_frontal, w, h)
cv2.resizeWindow(window_topdown, w, h)
cv2.moveWindow(window_frontal, 0, 0)
cv2.moveWindow(window_topdown, w + 80, 0)

clicked_point = {
    window_frontal: None,
    window_topdown: None
}

def callback_frontal(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDOWN:
        print(f"Frontal view: click at ({x}, {y}), depth = {sample.depth_map[y, x]:.3f} meter")
        clicked_point[window_frontal] = (x, y)

    if event == cv2.EVENT_RBUTTONDOWN:
        print(f"Frontal view: removed click.")
        clicked_point[window_frontal] = None

def callback_topdown(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDOWN:
        print(f"Topdown view: click at ({x}, {y})")
        clicked_point[window_topdown] = (x, y)
    if event == cv2.EVENT_RBUTTONDOWN:
        print(f"Topdown view: removed click.")
        clicked_point[window_topdown] = None

def draw_point(image: OpenCVIntImageType, point: tuple[int, int], color: tuple[int, int, int] = (0, 255, 0)):
    cv2.circle(image, point, point_size, color, point_thickness, line_type)


# color_frontal = (0, 255, 255)
color_topdown = (255, 255, 0)
color_mixed = (255, 0, 255)

cv2.setMouseCallback(window_frontal, callback_frontal)
cv2.setMouseCallback(window_topdown, callback_topdown)

while True:
    image = ImageConverter.from_numpy_int_format(sample.image_left).image_in_opencv_format

    image_frontal = image.copy()
    image_topdown = ImageConverter.from_numpy_format(image_rgb_float).image_in_opencv_format  

    if clicked_point[window_frontal] is not None:
        # draw_point(image_frontal, clicked_point[window_frontal], color_frontal)

        u, v = clicked_point[window_frontal]
        image_coordinates = np.array([u, v]).reshape((1, 2))
        depth_values = np.array([sample.depth_map[v, u]])
        p_C = unproject_onto_depth_values(image_coordinates, depth_values, sample.camera_intrinsics)
        p_W = transform_points(X_W_C, p_C).squeeze()

        X_W_C2 = camera_pose_top_down
        X_C2_W = np.linalg.inv(camera_pose_top_down)
        image_point_in_camera2 = project_points_to_image_plane(transform_points(X_C2_W, p_W), sample.camera_intrinsics).squeeze()
        image_point_in_camera2_int = np.rint(image_point_in_camera2).astype(int)
        # draw_point(image_topdown, tuple(image_point_in_camera2_int), color_frontal)

        Z = np.array([1, 0, 0])

        if clicked_point[window_topdown] is not None:
            vector_in_camera2 = image_point_in_camera2 - np.array(clicked_point[window_topdown])

            arrow_start = clicked_point[window_topdown]
            arrow_end = tuple(image_point_in_camera2_int)
            arrow_length = np.linalg.norm(vector_in_camera2)
            cv2.arrowedLine(image_topdown, arrow_start, arrow_end, color_mixed, 3, line_type, 0, 25.0 / arrow_length)

            v_C2 = np.array([vector_in_camera2[0], vector_in_camera2[1], 0])
            v_3d_C2 = v_C2 / np.linalg.norm(v_C2) # np.linalg.inv(sample.camera_intrinsics) @ v_C2
            v_3d_C2_homogeneous = np.array([v_3d_C2[0], v_3d_C2[1], v_3d_C2[2], 0])
            v_3d_W = (X_W_C2 @ v_3d_C2_homogeneous)[:3]
            
            Z = v_3d_W
        Y = np.array([0, 0, 1])
        X = np.cross(Y, Z)
        orientation = np.column_stack([X, Y, Z])
        position = p_W

        grasp_pose = np.identity(4)
        grasp_pose[:3, :3] = orientation
        grasp_pose[:3, 3] = position

        draw_pose(image_frontal, grasp_pose, sample.camera_intrinsics, X_W_C)
        draw_pose(image_topdown, grasp_pose, sample.camera_intrinsics, X_W_C2)


    if clicked_point[window_topdown] is not None:
        draw_point(image_topdown, clicked_point[window_topdown], color_mixed)

    cv2.imshow(window_frontal, image_frontal)
    cv2.imshow(window_topdown, image_topdown)
    key = cv2.waitKey(10)
    if key== ord('q'):
        cv2.destroyAllWindows()
        break